In [1]:
# Load libraries + setup

import sqlite3
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Load customer data

conn = sqlite3.connect("../data/project.db")

query = """
WITH last_date AS (
    SELECT MAX(InvoiceDate) AS max_date
    FROM transactions
),
customer_agg AS (
    SELECT
        "Customer ID" AS customer_id,
        COUNT(DISTINCT Invoice) AS frequency,
        SUM(Quantity * Price) AS monetary,
        MAX(InvoiceDate) AS last_purchase_date
    FROM transactions
    GROUP BY "Customer ID"
)
SELECT
    c.customer_id,
    c.frequency,
    c.monetary,
    CAST(julianday(l.max_date) - julianday(c.last_purchase_date) AS INT) AS recency_days,
    CASE
        WHEN julianday(l.max_date) - julianday(c.last_purchase_date) > 90 THEN 1
        ELSE 0
    END AS churned
FROM customer_agg c
CROSS JOIN last_date l;
"""

df = pd.read_sql_query(query, conn)
conn.close()

df.head()


,customer_id,frequency,monetary,recency_days,churned
0,12346,11,372.86,164,1
1,12347,2,1323.32,2,0
2,12348,1,222.16,73,0
3,12349,3,2671.14,42,0
4,12351,1,300.93,10,0


In [7]:
# Define features and train model

x = df[["frequency", "monetary"]]
y = df["churned"]

x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    class_weight="balanced"
)

rf.fit(x_train, y_train)

# evaluate at threshold = 0.5 (default)

y_pred_rf = rf.predict(x_test)
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

importances = pd.DataFrame({
    "feature": x.columns,
    "importance": rf.feature_importances_
}).sort_values(by="importance", ascending=False)

importances


[[428 148]
 [162 125]]
              precision    recall  f1-score   support

           0       0.73      0.74      0.73       576
           1       0.46      0.44      0.45       287

    accuracy                           0.64       863
   macro avg       0.59      0.59      0.59       863
weighted avg       0.64      0.64      0.64       863



,feature,importance
1,monetary,0.859098
0,frequency,0.140902


According to the feature importance chart above, this random forest seems to conclude that low spenders churn and high spenders do not, which ignores loyal low spenders and high spenders who disengage, leading to a weaker recall than the logistic regression model.

In [5]:
y_prob_rf = rf.predict_proba(x_test)[:, 1]

# Try the same threshold as logistic regression
threshold = 0.416
y_pred_rf_thresh = (y_prob_rf >= threshold).astype(int)

print(confusion_matrix(y_test, y_pred_rf_thresh))
print(classification_report(y_test, y_pred_rf_thresh))


[[425 151]
 [160 127]]
              precision    recall  f1-score   support

           0       0.73      0.74      0.73       576
           1       0.46      0.44      0.45       287

    accuracy                           0.64       863
   macro avg       0.59      0.59      0.59       863
weighted avg       0.64      0.64      0.64       863



I compared a linear and a non-linear model using the same behavioural features. The random forest did not improve recall and relied heavily on monetary value (0.86 importance), whereas the logistic regression provided better early-warning performance after threshold tuning. Given the comparable or better performance and greater interpretability, logistic regression was selected as the final model.